## TODO
 - [ ] Remover músicas instrumentais (elas não tem letra)
 - 

In [1]:
import numpy as np
import pandas as pd
from songs_analyzer.config import get_root_path
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB


In [2]:
def join_lyrics(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ', '.join(x))
    return df

def remove_unknown_genders(df):
    unknown_genders = ['L', 'T','J', 'K','Z', 'M', 'R', 'B', 'V', 'E', 'G', 'F', 'S', 'D', 'H', 'P', 'C', 'A']
    return df.query("gender not in @unknown_genders")

def remove_small_gender(df):
    gender_count = df.value_counts('gender')
    values = gender_count[gender_count.values < 50].index
    return df.query("gender not in @values")

def transform_title(df):
    df['title'] = df['title'].apply(lambda x: x[0] if isinstance(x, list) else x)
    return df

def remove_instrumental_music(df):
    values = ['Instrumental', '(Instrumental)', '[Instrumental]', "", "instrumental"]
    return df.query("lyric not in @values")

def remove_small_letters(df):
    df = df.query('lyric.str.len() > 100')
    return df[df.lyric.apply(lambda x: len(set(x.split())) > 40)]
    

In [19]:
df = (
    pd
    .read_json(get_root_path() / "data/raw/songs.json")
    .pipe(join_lyrics, ['lyric'])
    .pipe(remove_unknown_genders)
    .pipe(remove_small_gender)
    .pipe(transform_title)
    .pipe(remove_instrumental_music)
    .pipe(remove_small_letters)
    
)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(subset='lyric', keep="first", inplace=True)
df.head()

,title,lyric,gender,artist
0,My Heart Beats Like A Drum (Dam Dam Dam),"Eu fecho meu olhos, Fecho a porta, Não quero m...",Dance,ATC
1,"Broken, Beat & Scarred","Você levanta, Você cai, Você está derrubado, e...",Heavy Metal,Metallica
2,Good Grief,"Então, o que vocês, pequenos maníacos, gostari...",Indie,Bastille
3,Flaming Hot Cheetos,Às vezes eu sinto que eu só quero voltar para ...,Alternativo,Clairo
4,Flores para Iemanjá,"Eu vi de longe, Vindo do mar, As flores que eu...",Axé,Márcio Faraco


In [20]:
df.shape

(26405, 4)

In [21]:
X = df['lyric']
y = df['gender']

Talvez seja bom capturar mais músicas dos gêneros que saíram pouco e são conhecidos, como `forró`, `Sertanejo`, etc.

In [22]:
stop_words = stopwords.words('portuguese')
stemmer = PortugueseStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = TfidfVectorizer(
    analyzer=stemmed_words,
    stop_words=stop_words,
    max_features=5000
)
transformer = Pipeline(
    steps= [
        ('stemmer_tfidf', stem_vectorizer)
    ]
)

In [23]:
X_vector = transformer.fit_transform(X)

In [24]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X_vector)

NearestNeighbors(n_neighbors=10)

In [40]:
df.query('title == "')

,title,lyric,gender,artist
15508,De Quem É a Culpa?,"Exagerado, sim, Sou mais você que eu, Sobreviv...",Sertanejo,Marília Mendonça


In [41]:
letra = df.query('title == "De Quem É a Culpa?"').lyric.to_list()
songs = neigh.kneighbors(transformer.transform(letra), return_distance=False)
df.iloc[songs[0]]

,title,lyric,gender,artist
15508,De Quem É a Culpa?,"Exagerado, sim, Sou mais você que eu, Sobreviv...",Sertanejo,Marília Mendonça
9215,Céu Azul (Poesia Acústica #7),"[Mc Hariel], Liberdade Rennan da Penha, é Poes...",Hip Hop/Rap,Pineapple
2821,Céu Azul,"[MC Hariel], Liberdade Rennan da Penha, é Poes...",Funk,MC Kevin o Chris
9667,Estilo Vagabundo 3,Eu já não quero saber o que você pensa de mim ...,Hip Hop/Rap,MV Bill
1212,Meu Lugar,"Você já sabe que em qualquer lugar, Eu tô pens...",Reggae,Onze:20
7736,"Tô Bem, Tô Zen (part. Bella Angel)","Garoto, escute aqui, Eu sou aquela que sempre ...",Funk,Melody
5495,Wasted Times,Tempos desperdiçados que eu passei com outra p...,R&B,The Weeknd
22456,Dúvida Cruel,"Tô em casa me sentindo mal, já são onze e tal,...",Pagode,Dilsinho
12490,Quem Mandou Você Olhar,"Eu não sou de me entregar, Mas quem mandou me ...",Pagode,Ferrugem
15703,Tô Sem Você,"E você disse adeus pro meu coração, Sem querer...",Brega,João Gomes


In [26]:
df.query('title == "Blood"')

,title,lyric,gender,artist
12630,Blood,"Uma vela no peitoril da janela, Queimando bril...",Country,Allison Moorer
19369,Blood,Eu te odeio pelos sacrifícios que você fez por...,Heavy Metal,In This Moment


,title,lyric,gender,artist
27582,Só Você,"Demorei muito pra te encontrar, Agora eu quero...",Romântico,Fábio Jr.
20416,Eu Quero Só Você,"Ôôô (7x), Ôôô (7x), O que todos querem, É só s...",Axé,Chicabana
9552,Eu Quero Só Você,"O que todos querem, É só separar nóis dois, Vo...",Axé,Chicabana
19098,Banido Desbanido,"Banido, banido, banido, banido, Banido, banido...",Brega,Ednaldo Pereira
25095,Defekt,"Angst erwischt dich, angst besetzt dich, angst...",Industrial,Oomph!
22244,La gatta mammona,"Chi è stat?, Chi è stat?, Chi è stat?, Chi è s...",Funk Internacional,99 Posse
22254,Lamento Às Águas,"É niboboiá, Cabirê ô Yabá, É niboboiá, Cabirê ...",Axé,Os Tincoãs
23479,Namah Shivaya,"Om Namah Shivãya, Shivãya Namaha, Shivãya Nama...",Indie,Krishna Das
6483,Solo Quiero,"Seu olhar roubou meu coração, Eu gasto me obse...",Romântico,Johnny Sky
27225,I Just Want to Make Love to You,"Eu não quero que você seja nenhum escravo, Eu ...",Blues,Willie Dixon


In [7]:
decisiontree = DecisionTreeClassifier()

mlp = MLPClassifier(
    hidden_layer_sizes=(100,), 
    activation='relu',
    solver = 'adam',
    alpha=0.001
)

# clf = StackingClassifier(
#     estimators=[
#         ('decisiontree',decisiontree),
#         ('lr', lr),
#         ('mlp', mlp),
#         ('svc',svc)
#     ],
#     final_estimator=svc,
#     cv=10
# )

models = {
    'nb': MultinomialNB(),
    # 'lr': LogisticRegression(multi_class='multinomial', n_jobs=-1, max_iter=1000),
    'svc':SVC(probability=True, max_iter=1000),
    'decisiontree': DecisionTreeClassifier(),
    # 'mlp': mlp,
    # 'stacked': clf
}

In [8]:
results = {}
for model_name, model in models.items():
    reg = Pipeline(
            steps=[
                ('preprocessor', transformer),
                ('regressor', model)
            ]
        )
    scores = cross_validate(reg, X, y, scoring=[ 'accuracy'], cv=5)
    results[model_name] = {
        # 'f1': scores['test_f1'],
        'accuracy': scores['test_accuracy']
    }
    print(results)

{'nb': {'accuracy': array([0.31024531, 0.32233045, 0.32251082, 0.31282699, 0.32635757])}}


KeyboardInterrupt: 